In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sys
import pickle

In [11]:
df=pd.read_csv(r'C:\Users\Yasaman\avalanche_sat\output\WFJ2.csv')

In [12]:
df

,Unnamed: 0,MS_Water_daily,sum_up_3_diff,S5_daily,ISWR_h_daily,mean_lwc_3_diff,mean_lwc_2_diff,water_1_diff,RH_daily,prop_up,...,LWR_net,Qw_daily,pAlbedo_daily,HS,sum_up,station_code,aspect,profile_time,datum,probability_wet_AvD_model_2001_2022
0,0,0.00000,-0.110435,0.000000,101.004875,-0.110435,-0.025997,0.000,94.0625,0.000000,...,-90.981,10.879875,0.897750,35.30,0.000000,WFJ2,NaN,2021-11-05 15:00:00,2021-11-05,0.002
1,0,0.03625,0.087271,0.003750,171.982125,0.027634,0.053632,1.750,37.6750,0.619417,...,-120.317,34.732125,0.830000,32.63,0.113269,WFJ2,NaN,2021-11-06 15:00:00,2021-11-06,0.030
2,0,0.10400,0.322808,0.011375,187.553000,0.178597,0.178597,3.699,16.9375,0.535545,...,-123.033,47.803000,0.770875,30.51,0.322808,WFJ2,NaN,2021-11-07 15:00:00,2021-11-07,0.084
3,0,0.00000,0.000000,0.000000,170.344125,0.000000,-0.053632,-5.449,57.9375,0.000000,...,-56.731,38.469125,0.798000,29.21,0.000000,WFJ2,NaN,2021-11-08 15:00:00,2021-11-08,0.015
4,0,0.01825,-0.008180,0.002125,173.851625,0.004578,-0.120387,1.652,27.8000,0.378499,...,-72.471,45.226625,0.777000,28.38,0.105089,WFJ2,NaN,2021-11-09 15:00:00,2021-11-09,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0,11.33475,0.482155,1.168000,195.623250,0.194346,0.602554,-14.641,95.0500,1.000000,...,-66.084,49.498250,0.747750,35.57,4.122244,WFJ2,NaN,2022-05-28 00:00:00,2022-05-28,0.401
205,0,6.15225,-2.353416,0.704250,230.413875,-1.253823,-1.295272,-32.044,94.3000,1.000000,...,-99.128,55.163875,0.780625,29.11,1.746645,WFJ2,NaN,2022-05-29 15:00:00,2022-05-29,0.013
206,0,6.06350,-0.546146,0.730250,203.702375,-0.469749,-0.195483,30.529,92.2375,1.000000,...,-81.664,52.327375,0.765750,26.99,3.722748,WFJ2,NaN,2022-05-30 18:00:00,2022-05-30,0.031
207,0,7.30975,0.216551,0.910500,223.211750,0.146814,1.246373,14.876,81.8375,1.000000,...,-64.629,69.086750,0.734250,26.66,3.921614,WFJ2,NaN,2022-05-31 15:00:00,2022-05-31,0.297
